In [116]:
from collections import deque


class VictoryFlag(Exception):
    pass


class ElfDead(Exception):
    pass



class Dude:
    node = None
    enemy = None
    friends = None
    hp = None
    power = None

    def __init__(self, node):
        assert(node.dude is None)
        self.node = node
        node.dude = self
        self.friends.add(self)

    def move(self):
        if not self.enemy.friends:
            raise VictoryFlag()

        # Don't move if next to enemy
        if self._near_enemy(self.node):
            return
        
        next_node = self._find_move()
        if next_node:
            next_node.dude = self
            self.node.dude = None
            self.node = next_node

    def attack(self):
        enemies = [n.dude for n in self.node.links if isinstance(n.dude, self.enemy)]
        if not enemies:
            return
        target = sorted(enemies, key=lambda e: (e.hp, e.node.pos))[0]
        target.hp -= self.power
        if target.hp < 1:
            target.die()

    def die(self):
        self.friends.remove(self)
        self.node.dude = None

            
    @classmethod
    def _near_enemy(cls, node):
        return any(isinstance(n.dude, cls.enemy) for n in node.links)

    def _find_move(self):
        # Breadth-first search to find an empty space near an enemy
        seen = {self.node}
        new_fringe = [(None, self.node)]
        fringe = []
        donezo = []
        while not donezo:
            if not new_fringe:
                return None
            fringe, new_fringe = new_fringe, []
            for prev_start, node in fringe:
                for next_node in node.links:
                    start = prev_start or next_node
                    if next_node.dude or next_node in seen:
                        continue
                    if self._near_enemy(next_node):
                        donezo.append((next_node.pos, start.pos, start))
                    seen.add(next_node)
                    new_fringe.append((start, next_node))
        return sorted(donezo)[0][2]


class Goblin(Dude):
    friends = None
    enemy = None
    power = 3
    hp = 200
    
class Elf(Dude):
    friends = None
    enemy = None
    power = 3
    hp = 200

    def die(self):
        super().die()
        raise ElfDead()
    
class Node:
    dude = None
    links = None
    pos = None
    
    def __init__(self, row, col, top_node, left_node):
        self.links = []
        self.pos = (row, col)
        if top_node:
            self.links.append(top_node)
            top_node.links.append(self)
        if left_node:
            self.links.append(left_node)
            left_node.links.append(self)
            
            
class Board:
    turns = None
    nodes = {}

    def __init__(self, filename):
        self.turns = 0
        Goblin.friends = set()
        Elf.friends = set()
        Goblin.enemy = Elf
        Elf.enemy = Goblin

        parse_node = {
            '.': (Node, None),
            '#': (None, None),
            'G': (Node, Goblin),
            'E': (Node, Elf),
        }

        with open(filename) as infile:
            row = None
            for r, l in enumerate(infile):
                prev_row = row
                row = []
                for c, rune in enumerate(l.strip()):
                    node = None
                    node_class, dude_class = parse_node[rune]
                    if node_class:
                        node = node_class(r, c, prev_row[c], row[-1])
                        self.nodes[(r,c)] = node
                    if dude_class:
                        dude_class(node)
                    row.append(node)
      
    def tick(self):
        dudes = Elf.friends | Goblin.friends
        for d in sorted(dudes, key=lambda d: d.node.pos):
            if d.hp < 1:
                continue
            try:
                d.move()
            except VictoryFlag:
                return True
            
            try:
                d.attack()
            except ElfDead:
                # Hacky, i know..
                if Elf.power > 3:
                    raise
        self.turns += 1
    
    def run(self, elf_power=3):
        Elf.power = elf_power
        while Elf.friends and Goblin.friends:
#             print(self.turns, len(Goblin.friends), len(Elf.friends))
            self.tick()
        return(self.turns * (sum(d.hp for d in (Elf.friends | Goblin.friends))))


In [118]:
b = Board('input.txt')
b.run()

269430

In [7]:
def debug():
    print(b.turns)
    for d in sorted(Goblin.friends | Elf.friends, key=lambda d: d.node.pos):
        print(type(d).__name__[0], d.node.pos, d.hp)

In [111]:
elf_power = 3
while elf_power <= 200:
    elf_power += 1
    print(elf_power)
    b = Board("input.txt")
    try:
        b.run(elf_power)
    except ElfDead:
        continue
    break


4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
55160
